<a href="https://colab.research.google.com/github/jobeda123/Glycation_SVM_CKSAAP/blob/master/cksaap_glycation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
############ Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from sklearn.utils import class_weight
from keras.utils import plot_model
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import matthews_corrcoef
import random
from sklearn.model_selection import KFold
from sklearn import tree
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import svm


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
##### For Positive data ----- import CKSAAP where k=5

positive_cksaap= pd.read_csv('/content/drive/My Drive/Dataset_for_Glycation/positive_CKSAAP.csv')

print(positive_cksaap.shape)


(3065, 2401)


In [ ]:
################## Spliting into train and test in positive data

x_train_1,x_test_1=train_test_split(positive_cksaap,test_size=0.2)
print(len(x_train_1))
print(len(x_test_1))


2452
613


In [ ]:
##### For Negative data ----- import CKSAAP where k=5

negative_cksaap= pd.read_csv('/content/drive/My Drive/Dataset_for_Glycation/negative_CKSAAP.csv')

print(negative_cksaap.shape)

(20715, 2401)


In [ ]:
################## Spliting into train and test  negative data

x_train_0,x_test_0=train_test_split(negative_cksaap,test_size=0.2)
print(len(x_train_0))
print(len(x_test_0))


16572
4143


In [ ]:
############# Label combined for train

y_train=[]
for i in range(len(x_train_1)):
  y_train.append(1)

#print(len(y))  

i=0
for i in range(len(x_train_0)):
  y_train.append(0)

#print(len(y))


#######  Feature combined for cksaap for train

a1=x_train_0.shape[0]+x_train_1.shape[0]
b1=x_train_1.shape[1]

#print(a1)
#print(b1)
#print(c1)

cksaap_train=np.zeros((a1,b1))
print(cksaap_train.shape)
cksaap_train=np.concatenate((x_train_1,x_train_0))

print(len(cksaap_train))



(19024, 2401)
19024


In [ ]:
############# Label combined for test

y_test=[]
for i in range(len(x_test_1)):
  y_test.append(1)

#print(len(y))  

i=0
for i in range(len(x_test_0)):
  y_test.append(0)

#print(len(y))


#######  Feature combined for cksaap for train

a1=x_test_0.shape[0]+x_test_1.shape[0]
b1=x_test_1.shape[1]

#print(a1)
#print(b1)
#print(c1)

cksaap_test=np.zeros((a1,b1))
print(cksaap_test.shape)
cksaap_test=np.concatenate((x_test_1,x_test_0))

print(len(cksaap_test))



(4756, 2401)
4756


In [ ]:
####################  To Solve Imbalance Data for cksaap


counter = Counter(y_train)
print("Before using SMOTE: ",counter)

# transform the dataset
oversample = SMOTE()
x_train_cksaap, y_train_cksaap = oversample.fit_resample(cksaap_train, y_train)


# summarize the new class distribution
counter = Counter(y_train_cksaap)
print("After using SMOTE:",counter)

Before using SMOTE:  Counter({0: 16572, 1: 2452})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


After using SMOTE: Counter({1: 16572, 0: 16572})


In [ ]:
#################### SVM model for cksaap with balance data

"""

############ Scaling features

#min_max_scaler = preprocessing.MinMaxScaler()
#cksaap_minmax = min_max_scaler.fit_transform()
#print(onehot_features_minmax )

######## Split data 

#x_train,x_test,y_train,y_test=train_test_split(onehot_features_minmax ,y_onehot_balance,test_size=0.2)


clf = SVC(C=16)
clf.fit(x_train_cksaap, y_train_cksaap)
y_predict=clf.predict(cksaap_test)
acc=accuracy_score(y_test,y_predict)
confusion=confusion_matrix(y_test,y_predict)
print("Confusion Matrix: ",confusion)
TP=confusion[1,1]
print("TP:",TP)
TN=confusion[0,0]
print("TN: ",TN)
FP=confusion[0,1]
print("FP: ",FP)
FN=confusion[1,0]
print("FN: ",FN)
a1=(TN+TP)/(float)(TN+TP+FN+FP)
SN=TP/(float)(TP+FN)
print("Sensivity: ",SN)
SP=TN/(float)(TN+FP)
print("Specificity: ",SP)
mcc= matthews_corrcoef(y_test,y_predict)
print("MCC: ",mcc)
print("Accuracy from quation: ",a1)
print("Accuracy from model: ",acc)
"""

'\n\n############ Scaling features\n\n#min_max_scaler = preprocessing.MinMaxScaler()\n#cksaap_minmax = min_max_scaler.fit_transform()\n#print(onehot_features_minmax )\n\n######## Split data \n\n#x_train,x_test,y_train,y_test=train_test_split(onehot_features_minmax ,y_onehot_balance,test_size=0.2)\n\n\nclf = SVC(C=16)\nclf.fit(x_train_cksaap, y_train_cksaap)\ny_predict=clf.predict(cksaap_test)\nacc=accuracy_score(y_test,y_predict)\nconfusion=confusion_matrix(y_test,y_predict)\nprint("Confusion Matrix: ",confusion)\nTP=confusion[1,1]\nprint("TP:",TP)\nTN=confusion[0,0]\nprint("TN: ",TN)\nFP=confusion[0,1]\nprint("FP: ",FP)\nFN=confusion[1,0]\nprint("FN: ",FN)\na1=(TN+TP)/(float)(TN+TP+FN+FP)\nSN=TP/(float)(TP+FN)\nprint("Sensivity: ",SN)\nSP=TN/(float)(TN+FP)\nprint("Specificity: ",SP)\nmcc= matthews_corrcoef(y_test,y_predict)\nprint("MCC: ",mcc)\nprint("Accuracy from quation: ",a1)\nprint("Accuracy from model: ",acc)\n'

In [ ]:
############### Merge train and test for 10-fold cross validation

"""
a=x_train_cksaap.shape[0]+cksaap_test.shape[0]
b=x_train_cksaap.shape[1]

#print(a1)
#print(b1)

all_cksaap=np.zeros((a,b))
print(all_cksaap.shape)
all_cksaapt=np.concatenate((x_train_cksaap,cksaap_test))

print(len(all_cksaap))

all_y=np.concatenate((y_train_cksaap,y_test))
print(len(all_y))

"""

'\na=x_train_cksaap.shape[0]+cksaap_test.shape[0]\nb=x_train_cksaap.shape[1]\n\n#print(a1)\n#print(b1)\n\nall_cksaap=np.zeros((a,b))\nprint(all_cksaap.shape)\nall_cksaapt=np.concatenate((x_train_cksaap,cksaap_test))\n\nprint(len(all_cksaap))\n\nall_y=np.concatenate((y_train_cksaap,y_test))\nprint(len(all_y))\n\n'

In [ ]:
################## KFold Cross Validation approach


clf = SVC(C=16)
kf = KFold(n_splits=10,shuffle=True)
kf.split(x_train_cksaap)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
sn_model = []
sp_model = []
f1_model = []
mcc_model= []

 
# Iterate over each train-test split
for train_index, test_index in kf.split(x_train_cksaap):
    print("************************ Model***********************")
    # Split train-test
    X_train, X_test = x_train_cksaap[train_index], x_train_cksaap[test_index]
    y_train, y_test = y_train_cksaap[train_index], y_train_cksaap[test_index]
    # Train the model
    model = clf.fit(X_train, y_train)
    # Append to accuracy_model the accuracy of the model
    y_pred=model.predict(X_test)
    accuracy_model.append(accuracy_score(y_test, y_pred)*100)
    confusion=confusion_matrix(y_test,y_pred)
    print("Confusion Matrix: ",confusion)
    TP=confusion[1,1]
    print("TP:",TP)
    TN=confusion[0,0]
    print("TN: ",TN)
    FP=confusion[0,1]
    print("FP: ",FP)
    FN=confusion[1,0]
    print("FN: ",FN)
    a1=(TN+TP)/(float)(TN+TP+FN+FP)
    SN=TP/(float)(TP+FN)
    sn_model.append(SN)
    print("Sensivity: ",SN)
    SP=TN/(float)(TN+FP)
    sp_model.append(SP)
    print("Specificity: ",SP)
    mcc= matthews_corrcoef(y_test,y_pred)
    mcc_model.append(mcc)
    print("MCC: ",mcc)
    print("Accuracy from quation: ",a1)
    f1_model.append(f1_score(y_test, y_pred))
    print("**************************************************")

# Print the accuracy    
print("Accuracy:",accuracy_model)
print("Sensitivity:",sn_model)
print("Specificity:",sp_model)
print("F-1 Score:",f1_model)
print("MCC Score:",mcc_model)



a=sum(accuracy_model)/len(accuracy_model)
print("Average Accuracy: ",a)
b=sum(sn_model)/len(accuracy_model)
print("Average Sensitivity: ",b)
c=sum(sp_model)/len(accuracy_model)
print("Average Specificity: ",c)
d=sum(f1_model)/len(accuracy_model)
print("Average f-1 Score: ",d)
e=sum(mcc_model)/len(accuracy_model)
print("Average MCC Score: ",e)



************************ Model***********************
Confusion Matrix:  [[1359  263]
 [   0 1693]]
TP: 1693
TN:  1359
FP:  263
FN:  0
Sensivity:  1.0
Specificity:  0.8378545006165228
MCC:  0.8515856944919159
Accuracy from quation:  0.9206636500754147
**************************************************
************************ Model***********************
Confusion Matrix:  [[1344  294]
 [   0 1677]]
TP: 1677
TN:  1344
FP:  294
FN:  0
Sensivity:  1.0
Specificity:  0.8205128205128205
MCC:  0.8355374200564336
Accuracy from quation:  0.9113122171945701
**************************************************
************************ Model***********************
Confusion Matrix:  [[1352  286]
 [   0 1677]]
TP: 1677
TN:  1352
FP:  286
FN:  0
Sensivity:  1.0
Specificity:  0.8253968253968254
MCC:  0.8397263448763573
Accuracy from quation:  0.9137254901960784
**************************************************
************************ Model***********************
Confusion Matrix:  [[1363  257]
 [   

In [ ]:
################# Visualize accuracy for each iteration
 
"""
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Iter1','Iter2','Iter3','Iter4','Iter5','Iter6','Iter7','Iter8','Iter9','Iter10'],y="Scores",data=scores)
plt.show()
sns.set()

"""

'\nscores = pd.DataFrame(accuracy_model,columns=[\'Scores\'])\n \nsns.set(style="white", rc={"lines.linewidth": 3})\nsns.barplot(x=[\'Iter1\',\'Iter2\',\'Iter3\',\'Iter4\',\'Iter5\',\'Iter6\',\'Iter7\',\'Iter8\',\'Iter9\',\'Iter10\'],y="Scores",data=scores)\nplt.show()\nsns.set()\n\n'

In [ ]:
"""
accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes)
print('F1 score: %f' % f1)
"""

"\naccuracy = accuracy_score(testy, yhat_classes)\nprint('Accuracy: %f' % accuracy)\n# precision tp / (tp + fp)\nprecision = precision_score(testy, yhat_classes)\nprint('Precision: %f' % precision)\n# recall: tp / (tp + fn)\nrecall = recall_score(testy, yhat_classes)\nprint('Recall: %f' % recall)\n# f1: 2 tp / (2 tp + fp + fn)\nf1 = f1_score(testy, yhat_classes)\nprint('F1 score: %f' % f1)\n"

In [ ]:

"""
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score

clf = svm.SVC(C=16)
scores = cross_val_score(clf, x_train_cksaap, y_train_cksaap, cv=5)
print(scores)

"""

'\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.metrics import recall_score\n\nclf = svm.SVC(C=16)\nscores = cross_val_score(clf, x_train_cksaap, y_train_cksaap, cv=5)\nprint(scores)\n\n'